<a href="https://colab.research.google.com/github/martinpius/Applied-Predictive-Modeling2/blob/master/Bayesian_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
#Setup the colab environment
from google.colab import drive
try:
  drive.mount('/content/drive', force_remount = True)
  import tensorflow as tf
  COLAB = True
  print(f"You are using Google Colab environment with tensorflow version--->{tf.__version__}")
except:
  COLAB = False
  print("You are not connected:")

Mounted at /content/drive
You are using Google Colab environment with tensorflow version--->2.3.0


In [90]:
#Format the time in a nice readable form.
def time_setter(w):
  h = int(w/(60*60))
  m = int(w%(60*60)/60)
  s = int(w%60)
  return f"{h}:{m:>02}: {s:>05.2f}"

In [91]:
#Install optimization pack
!pip install bayesian-optimization

In [92]:
#Import necessary packs
import time
import os
import warnings
import pandas as pd
import numpy as np
import tensorflow as tf
from scipy.stats import zscore
from bayes_opt import BayesianOptimization
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, PReLU, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow.keras



In [93]:
#Read in the data from url
emp = pd.read_csv("https://data.heatonresearch.com/data/t81-558/jh-simple-dataset.csv", na_values = ['NAN','?'])

In [94]:
display(emp.head())

,id,job,area,income,aspect,subscriptions,dist_healthy,save_rate,dist_unhealthy,age,pop_dense,retail_dense,crime,product
0,1,vv,c,50876.0,13.100000,1,9.017895,35,11.738935,49,0.885827,0.492126,0.071100,b
1,2,kd,c,60369.0,18.625000,2,7.766643,59,6.805396,51,0.874016,0.342520,0.400809,c
2,3,pe,c,55126.0,34.766667,1,3.632069,6,13.671772,44,0.944882,0.724409,0.207723,b
3,4,11,c,51690.0,15.808333,1,5.372942,16,4.333286,50,0.889764,0.444882,0.361216,b
4,5,kl,d,28347.0,40.941667,3,3.822477,20,5.967121,38,0.744094,0.661417,0.068033,a


In [95]:
emp.drop('id', axis = 1, inplace = True)

In [96]:
y =emp['product']

In [97]:
#Fill missing values
emp['income'] = emp['income'].fillna(emp['income'].median())

In [98]:
#Select continous features to statndardize
cols_num = emp.columns.drop(['job','area','subscriptions','product'])

In [99]:
#Get the z-scores
for col in cols_num:
  emp[col] = zscore(emp[col])

In [100]:
display(emp.head())

,job,area,income,aspect,subscriptions,dist_healthy,save_rate,dist_unhealthy,age,pop_dense,retail_dense,crime,product
0,vv,c,-0.607550,-0.664918,1,-0.048411,-0.215764,-0.314089,0.854321,0.079279,-0.465765,-1.120315,b
1,kd,c,0.338053,-0.207748,2,-0.266765,0.196869,-0.915161,1.394432,-0.075010,-1.445372,0.682945,c
2,pe,c,-0.184205,1.127906,1,-0.988286,-0.714362,-0.078604,-0.495957,0.850727,1.055205,-0.373087,b
3,11,c,-0.526467,-0.440815,1,-0.684488,-0.542432,-1.216347,1.124377,0.130709,-0.775115,0.466401,b
4,kl,d,-2.851675,1.638861,3,-0.955058,-0.473660,-1.017291,-2.116291,-1.772196,0.642739,-1.137090,a


In [101]:
emp.drop('product', inplace = True, axis = 1)

In [102]:
#Dummifies categorical vars
jobd = pd.get_dummies(emp['job'], prefix = 'job')


In [103]:
aread = pd.get_dummies(emp['area'], prefix = 'area')

In [104]:
emp.drop(['job','area'],axis = 1, inplace = True)

In [105]:
display(emp.head())

,income,aspect,subscriptions,dist_healthy,save_rate,dist_unhealthy,age,pop_dense,retail_dense,crime
0,-0.607550,-0.664918,1,-0.048411,-0.215764,-0.314089,0.854321,0.079279,-0.465765,-1.120315
1,0.338053,-0.207748,2,-0.266765,0.196869,-0.915161,1.394432,-0.075010,-1.445372,0.682945
2,-0.184205,1.127906,1,-0.988286,-0.714362,-0.078604,-0.495957,0.850727,1.055205,-0.373087
3,-0.526467,-0.440815,1,-0.684488,-0.542432,-1.216347,1.124377,0.130709,-0.775115,0.466401
4,-2.851675,1.638861,3,-0.955058,-0.473660,-1.017291,-2.116291,-1.772196,0.642739,-1.137090


In [106]:
emp = pd.concat([emp, jobd, aread], axis = 1)

In [107]:
display(emp.head())

,income,aspect,subscriptions,dist_healthy,save_rate,dist_unhealthy,age,pop_dense,retail_dense,crime,job_11,job_al,job_am,job_ax,job_bf,job_by,job_cv,job_de,job_dz,job_e2,job_f8,job_gj,job_gv,job_kd,job_ke,job_kl,job_kp,job_ks,job_kw,job_mm,job_nb,job_nn,job_ob,job_pe,job_po,job_pq,job_pz,job_qp,job_qw,job_rn,job_sa,job_vv,job_zz,area_a,area_b,area_c,area_d
0,-0.607550,-0.664918,1,-0.048411,-0.215764,-0.314089,0.854321,0.079279,-0.465765,-1.120315,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
1,0.338053,-0.207748,2,-0.266765,0.196869,-0.915161,1.394432,-0.075010,-1.445372,0.682945,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,-0.184205,1.127906,1,-0.988286,-0.714362,-0.078604,-0.495957,0.850727,1.055205,-0.373087,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0
3,-0.526467,-0.440815,1,-0.684488,-0.542432,-1.216347,1.124377,0.130709,-0.775115,0.466401,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,-2.851675,1.638861,3,-0.955058,-0.473660,-1.017291,-2.116291,-1.772196,0.642739,-1.137090,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [108]:
x = emp.values

In [109]:
x[0:10]

array([[-0.60754957, -0.66491815,  1.        , -0.04841068, -0.21576413,
        -0.31408854,  0.85432106,  0.07927915, -0.46576475, -1.12031509,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  1.        ,  0.        ,  0.        ,  0.        ,
         1.        ,  0.        ],
       [ 0.33805295, -0.20774798,  2.        , -0.26676549,  0.19686897,
        -0.91516076,  1.39443237, -0.07501047, -1.44537236,  0.68294487,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.     

In [110]:
y = pd.get_dummies(y).values

In [111]:
y[0:10]

array([[0, 1, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0],
       [0, 1, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0]], dtype=uint8)

In [141]:
#Creating a helper function for hyperparameters turning
def nnt_helper(dropout,neuronsRate, neuronsCut):
  layers = 0
  model = Sequential()
  neurons_num = int(5000 * neuronsRate)
  
  while neurons_num > 50 and layers < 10:
    if layers == 0:
      model.add(Dense(neurons_num, input_dim = x.shape[1], kernel_initializer = 'random_normal', activation = PReLU()))
    else:
      model.add(Dense(neurons_num, activation = PReLU(), kernel_initializer = 'random_normal'))
    layers+=1
    model.add(Dropout(dropout))
    neurons_num = (neurons_num *neuronsCut)
    model.add(Dense(y.shape[1], activation = 'softmax'))
  return model

    


In [127]:
#Display the model prototype:
mymodel = nnt_helper(dropout = 0.25, neuronsRate=0.1, neuronsCut= 0.2)

In [128]:
mymodel.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_18 (Dense)             (None, 500)               24500     
_________________________________________________________________
dropout_9 (Dropout)          (None, 500)               0         
_________________________________________________________________
dense_19 (Dense)             (None, 7)                 3507      
_________________________________________________________________
dense_20 (Dense)             (None, 100)               900       
_________________________________________________________________
dropout_10 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_21 (Dense)             (None, 7)                 707       
Total params: 29,614
Trainable params: 29,614
Non-trainable params: 0
__________________________________________________

In [142]:
#Building the model
def my_turned_nnt(dropout, lr, neuronsPct,neuronsDrop):
  SPLITS = 2#Ten folds cross validations
  boots_samples = StratifiedShuffleSplit(n_splits = SPLITS, test_size = 0.1)
  av_epochs = []
  av_logloss = []
  count = 0
  #Iterate through the bootstrap samples:
  for train, test in boots_samples.split(x, y):
    start_time = time.time()
    count+=1
    x_train = x[train]
    x_test = x[test]
    y_train = y[train]
    y_test = y[test]

    #Call and training our nnt using the above helper's function:
    model = nnt_helper(dropout, neuronsPct,neuronsDrop)
    model.compile(loss = 'categorical_crossentropy')
    info = EarlyStopping(monitor = 'val_loss', patience = 100, min_delta = 1e-3, verbose = 1, restore_best_weights = True)
    model.fit(x_train,y_train, validation_data = (x_test, y_test), verbose = 0, epochs = 1000, callbacks = [info])
    epoc = info.stopped_epoch
    av_epochs.append(epoc)
    pred = model.predict(x_test)
    y_match = np.argmax(y_test, axis = 1)
    loss_score = metrics.log_loss(y_match, pred)
    av_logloss.append(loss_score)
    mean1 = statistics.mean(av_logloss)
    mean2 = statistics.mean(av_epochs)
    mean_dev = statistics.pstdev(av_logloss)
    time_elapse = time.time() - start_time
  tensorflow.keras.backend.clear_session()
  return -mean1
print(my_turned_nnt(dropout = 0.2, lr = 1e-3,neuronsPct =0.1,neuronsDrop = 0.2))


Restoring model weights from the end of the best epoch.
Epoch 00146: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00133: early stopping
-0.6586273719184101


In [143]:
#Bayesian optimization
from bayes_opt import BayesianOptimization

In [144]:
warnings.filterwarnings('ignore', category = RuntimeWarning)


In [145]:
# Hyper parameters bounding spaces
import logging, os
logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

Param_bound = {'dropout': (0.0, 0.499),
           'lr': (0.0, 0.1),
           'neuronsPct': (0.01, 1),
           'neuronsDrop': (0.01, 1)
          }

optimizer = BayesianOptimization(
    f=my_turned_nnt,
    pbounds=Param_bound,
    verbose=2,  
    random_state=1,
)

start_time = time.time()
optimizer.maximize(init_points=5, n_iter=10,)
time_took = time.time() - start_time

print(f"Total runtime: {time_setter(time_took)}")
print(optimizer.max)

|   iter    |  target   |  dropout  |    lr     | neuron... | neuron... |
-------------------------------------------------------------------------
Restoring model weights from the end of the best epoch.
Epoch 00105: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00114: early stopping
|  1        | -0.6574   |  0.2081   |  0.07203  |  0.01011  |  0.3093   |
Restoring model weights from the end of the best epoch.
Epoch 00126: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00133: early stopping
|  2        | -0.7776   |  0.07323  |  0.009234 |  0.1944   |  0.3521   |
Restoring model weights from the end of the best epoch.
Epoch 00105: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00124: early stopping
|  3        | -0.9871   |  0.198    |  0.05388  |  0.425    |  0.6884   |
Restoring model weights from the end of the best epoch.
Epoch 00130: early stopping
Restoring model weights from the end of th

ValueError: ignored